In [153]:
import os
import sys
import glob
import json
import requests
import pandas as pd
import pandas_gbq as gbq
from pathlib import Path
from IPython.core.debugger import set_trace
from tqdm.auto import tqdm
tqdm.pandas()

# Bigquery config

In [154]:
proj_id = 'nlp-project-253707'
table_id = 'news_dataset.news'

# Tagged news from Bigquery

In [183]:
sql = '''
SELECT url
FROM `nlp-project-253707.news_dataset.news`
'''

tagged_news = gbq.read_gbq(sql, project_id=proj_id)
tagged_news_url = list(tagged_news.url)

# check uniqueness of urls
assert len(tagged_news_url) == len(set(tagged_news_url)), 'url duplicated'

In [184]:
len(tagged_news)

8500

# Listing texts & urls to tag

In [176]:
def extract_to_tag(urls_asis, n_tag=500, where='newsdata/downloaded/*.json'):
    '''
    n_tag=500: uclassify가 허용하는 최대 text수가 500개임
    '''
    
    urls = []
    texts = []
    
    n = 0
    for fname in glob.glob(where):
        js = json.loads(Path(fname).read_text())

        if js['url'] not in urls_asis:        
            texts.append(js['text'])
            urls.append(js['url'])
            n += 1
            print('\r{}'.format(n), end='')
            if n == n_tag: break
                
    return texts, urls

In [177]:
texts, urls = extract_to_tag(tagged_news_url)

500

# Classification using uClassify
https://www.uclassify.com

In [178]:
apikeys = {
            'gem763': 'YbEhDr9hAYmd', 
    'projectester01': 'PUcCZkyO5wPs', # google 로그인해야함
    'projectester02': 'f2ISRvxphKf2',
    'projectester03': 'ebAUBLTIcdna',
    'projectester04': 'UTuQWyexBdOP',
    'projectester05': 'nfsPpO8ivVaR',
    'projectester06': '4659gHVFX8CQ',
    'projectester07': 'UljdQT1tzeVs',
    'projectester08': 'RWh9JGLYRho5',
    'projectester09': 'EHjhm6nONJv4',
    'projectester10': 'QZlAZF62SaEc',
    'projectester11': 'zxJQoBwzi1Bw',
    'projectester12': 'uZRjYK6Vx5aC',
    'projectester13': 'Vjk2T02S0Mvz',
    'projectester14': 'Z46sXa4fYKyO',
    'projectester15': 'EwLxqluy7WcY',
    'projectester16': 'NCVDU2DMxEXB',
    'projectester17': 'xUMOMxOIoWUY',
    'projectester18': 'B93XpEXzZsfa',
    'projectester19': 'vLuLVnQ4sCmM',
    'projectester20': 'ACIHqZxBXEBO',
}

In [179]:
def uclassify(_id, texts):
    headers = {'Authorization':'Token {}'.format(apikeys[_id])}
    endpoint = 'https://api.uclassify.com/v1/uclassify/IAB Taxonomy v2/classify'
    data = json.dumps({'texts': texts})

    return requests.post(endpoint, headers=headers, data=data)


def uclassify_response_to_dataframe(u_resp, urls, texts):
    records = []
    updated_at = str(pd.Timestamp.utcnow())

    for i, resp in enumerate(tqdm(u_resp.json())):
        cl = max(resp['classification'], key=lambda cand:cand['p'])    
        records.append({
            'url': urls[i], 
            'text': texts[i], 
            'uclass': cl['className'].split('_')[0], 
            'prob': cl['p'], 
            'updated_at': updated_at, 
        })

    return pd.DataFrame(records)

In [180]:
response = uclassify('projectester16', texts)

In [181]:
df = uclassify_response_to_dataframe(response, urls, texts); df

,prob,text,uclass,updated_at,url
0,0.975307,Far-right extremists are celebrating Boris Joh...,news and politics,2019-09-27 13:00:30.572820+00:00,https://independent.co.uk/news/uk/politics/bor...
1,0.543303,"RICHMOND, Va. (AP) - A new study has good news...",travel,2019-09-27 13:00:30.572820+00:00,https://washingtontimes.com/news/2019/sep/11/r...
2,0.892465,The surprise disclosure that agents from the F...,news and politics,2019-09-27 13:00:30.572820+00:00,https://wsj.com/articles/laptop-may-include-th...
3,0.998915,BARCELONA (Reuters) - With Lionel Messi still ...,sports,2019-09-27 13:00:30.572820+00:00,https://in.reuters.com/article/soccer-spain-pr...
4,0.216480,Two days after Ottawa Coun. Rick Chiarelli say...,news and politics,2019-09-27 13:00:30.572820+00:00,https://cbc.ca/news/canada/ottawa/chiarelli-in...
5,0.930958,© Reuters.\n\n(Bloomberg) -- U.S. stocks erase...,business and finance,2019-09-27 13:00:30.572820+00:00,https://investing.com/news/stock-market-news/s...
6,0.819528,Not My Job: 'Supremes' Singer Mary Wilson Gets...,pop culture,2019-09-27 13:00:30.572820+00:00,https://npr.org/2019/09/07/758276020/not-my-jo...
7,0.999899,Novartis AG's Sandoz unit said Wednesday it wa...,medical health,2019-09-27 13:00:30.572820+00:00,https://cbc.ca/news/health/zantac-1.5288242
8,0.992406,You've no doubt worked hard to take care of yo...,personal finance,2019-09-27 13:00:30.572820+00:00,https://news.yahoo.com/warren-buffett-apos-inv...
9,0.999260,"In theory, it might not seem like the most bli...",books and literature,2019-09-27 13:00:30.572820+00:00,https://theguardian.com/film/2019/sep/06/the-p...


# Bigquery update

In [182]:
gbq.to_gbq(df, table_id, project_id=proj_id, if_exists='append')

1it [00:10, 10.48s/it]


In [137]:
sql = '''
SELECT *
FROM `nlp-project-253707.news_dataset.news`
'''

test = gbq.read_gbq(sql, project_id=proj_id)

In [218]:
test

,prob,text,uclass,updated_at,url
0,0.292673,\n\nChris Sharp of the Fairfax County Police D...,pets,2019-09-26,https://washingtonpost.com/local/public-safety...
1,0.472381,Dir: Roger Hinze and Michael William Miles. Fe...,movies,2019-09-26,https://independent.co.uk/arts-entertainment/f...
2,0.621441,"Horror film ""IT Chapter Two"" stayed atop the N...",movies,2019-09-26,https://businesstimes.com.sg/life-culture/it-s...
3,0.993574,The 2019 Emmys are officially here.\n\nThe 71s...,movies,2019-09-26,https://yahoo.com/entertainment/emmys-2019-win...
4,0.293006,Eddie Murphy in Netflix’s Dolemite Is My Name ...,movies,2019-09-26,https://vulture.com/2019/09/tiff-screening-ven...
5,0.961838,This article originally appeared on VICE US.\n...,movies,2019-09-26,https://vice.com/en_asia/article/kz4bvw/mike-f...
6,0.434689,Image copyright Alamy Image caption Movie rema...,movies,2019-09-26,https://bbc.co.uk/news/newsbeat-49755700
7,0.670388,Angelina Jolie is a busy Hollywood actress and...,movies,2019-09-26,https://yahoo.com/entertainment/angelina-jolie...
8,0.511823,MEXICO CITY (Reuters) - Mexico's president on ...,sports,2019-09-26,https://businessinsider.com/mexico-president-s...
9,0.966641,MEXICO CITY (Reuters) - Mexican President Andr...,sports,2019-09-26,https://reuters.com/article/us-mexico-constell...


In [220]:
test[test.duplicated('url')]

,prob,text,uclass,updated_at,url
807,0.941261,But what if the way we think about paying our ...,business and finance,2019-09-26,https://politico.com/interactives/2019/how-to-...
